<a href="https://colab.research.google.com/github/drabgrey95/demandresponse/blob/main/hvac_cycling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this simulation uses minute-increments for all timing, degrees are in fahrenheit

To Do:


*2.  Incorporate windspeed + humidity*

Create DR strategy class

Fix 3rd stage cycling error

Allow for variable temperature curves

Allow for multiple zones from one RTU

Add zone to zone temperature interactions in initial test








In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import copy
import math
import statistics

In [ ]:
#!pip install streamlit
import streamlit as st

In [2]:
class hvac:
  def __init__(self, name, coolrate={0:0,1:-0.25,2:-0.5,3:-0.5}, heatrate={0:0,1:0.4}, ctimer=5, coolenergy={0:0,1:1,2:2,3:2}, heatenergy={0:0,1:4}):
    self.name = name
    self.history = {"compressor":[],
                    "heater":[]}
    self.ctimer = ctimer
    self.compressor = 0 #0=off, 1=1st stage, 2=in 2nd stage min runtime, 3=2nd stage past min runtime
    self.heater = 0 #0=off, 1=heating
    self.coolrate = coolrate
    self.heatrate = heatrate
    self.coolcycle = []
    self.heatcycle = []
    self.coolenergy = coolenergy
    self.heatenergy = heatenergy

  def __str__(self) -> str:
    return f"HVAC: {self.name} => Cool Cycle:{self.coolcycle}, Heat Cycle:{self.heatcycle}"

  def record(self):
    self.history["compressor"].append(self.compressor)
    self.history["heater"].append(self.heater)

  def reset(self):
    self.history["compressor"] = []
    self.history["heater"] = []
    self.coolcycle = []
    self.heatcycle = []
    self.compressor = 0
    self.heater = 0

  def energyUsage(self):
    total = 0
    for minute in self.history["compressor"]:
      total += self.coolenergy[minute]
    for minute in self.history["heater"]:
      total += self.heatenergy[minute]
    return total

  def DRenergy(self):
    total = 0
    for minute in range(60,120):
      total += self.coolenergy[self.history["compressor"][minute]]
      total += self.heatenergy[self.history["heater"][minute]]
    return total

h = hvac("RTU 1")

In [3]:
class zone:
  def __init__(self, name, sqfoot=0, temp=75.0, setpoints=[65,75], walls=(0.05,75.0,0.5), driftrate_factor=0.025) -> None:
    self.name = name
    self.history = []
    self.sqfoot = sqfoot
    self.temp = temp
    self.setpoints = setpoints
    self.walls = [walls[0],walls[1],walls[2]] #(rate of heat exchange between air and wall in degrees per minute, wall temperature)
    self.wallhistory = []
    self.driftrate_factor = driftrate_factor
        #this is not the drift rate exactly, but it is used to calculate the driftrate in the following way: driftrate = (outside temp - inside temp)*driftrate_factor
    self.dr = False

    self.stats = {"temp": temp,
                  "setpoints": setpoints,
                  "walls": walls,
                  }

  def __str__(self) -> str:
    return f"Zone: {self.name}, sq footage: {self.sqfoot}, temp: {self.temp}"

  def record(self):
    self.history.append(self.temp)
    self.wallhistory.append(self.walls[1])

  def reset(self):
    self.history = []
    self.wallhistory = []
    self.temp = self.stats["temp"]
    self.setpoints = self.stats["setpoints"]
    self.walls = [self.stats["walls"][0],self.stats["walls"][1],self.stats["walls"][2]]

  def wallChange(self, prevTemp):
    self.walls[1] = self.walls[1] - self.walls[0]*(self.walls[1]-prevTemp)

z = zone("ZONE_NAME")

In [4]:
class weather:
  def __init__(self, name="weather", temp=85.0, windspeed=0, humidity=0) -> None:
    self.name = name
    self.temp = temp
    self.windspeed = windspeed
    self.humidity = humidity

  def __str__(self) -> str:
    return f"Temp: {self.temp}, windspeed: {self.windspeed}, humidity: {self.humidity}"

w = weather()

In [5]:
class site:
  def __init__(self, name, weather=weather()) -> None:
    self.name = name
    self.system = {}
    self.weather = weather

  #def __str__(self) -> str:
  #  return f"Site name: {self.name} System: {self.system}"

  def add_rtu(self, rtu:hvac, zn:zone):
    self.system[zn] = rtu

#  def add_zone(self, rtu:hvac, zn:zone):
#    try:
#      self.system[rtu].append(zn)
#    except:
#      print("this RTU does not exist yet. Add it in using add_rtu")

  def change_weather(self, new_weather):
    self.weather = new_weather

s = site("SITE_NAME")

In [6]:
class DRprofile:
  def __init__(self, name="default", length=120, changes={0:(0,False)}):
    self.name = name
    self.length = length
    self.changes = changes


In [7]:
default = DRprofile()
stepup = DRprofile("stepup", 120, {0:(0,False), 60:(2,False)})
downupdown = DRprofile("downupdown", 180, {0:(0,False), 60:(2,False), 120:(-2,False)})
updownup = DRprofile("updownup", 180, {0:(0,False), 60:(-2,False), 120:(2,False)})
drdownupdown = DRprofile("downupdown", 180, {0:(0,False), 60:(2,True), 120:(-2,False)})
drupdownup = DRprofile("updownup", 180, {0:(0,False), 60:(-2,True), 120:(2,False)})

In [8]:
def regular_setpoint_control(zone, rtu):
  if zone.temp > zone.setpoints[1] or rtu.compressor in [1,2]:
    if rtu.compressor == 0:
      rtu.coolcycle = [1]*rtu.ctimer + [2]*rtu.ctimer + [3] #replace eventually with more nuanced 1st/2nd stage cycling strategy
    rtu.compressor = rtu.coolcycle.pop(0)
    if rtu.compressor == 3:
      rtu.coolcycle.append(3)
  elif rtu.compressor == 3 and zone.temp <= zone.setpoints[1]:
    rtu.compressor = 0
  elif zone.temp < zone.setpoints[0]: #add something to account for minimum heat cycle
    if rtu.heater == 0:
      rtu.heater = 1 #simple now, can replace with heatcycle
  elif zone.temp >= zone.setpoints[0]:
    rtu.heater = 0

In [ ]:
def dr_control(zone, rtu):
  drcycle = []
  wallContribution = zone.setpoints[1] - zone.walls[1] #assume walls reach temperature parity
  tempDiff = zone.setpoints[1] - zone.temp
  numCycles = 60/(rtu.ctimer*2) #fix so that 60 isn't default (make functiong giving length of DR event)
  coolpercycle = (statistics.mean(rtu.coolcycle[1],rtu.coolcycle[2]))



In [9]:
def simulation(site, profile, strategy=regular_setpoint_control, energy=True, graphs=False):
  #clear previous history
  for zone,rtu in site.system.items():
    zone.reset()
    rtu.reset()

  for minute in range(profile.length):
    for zone,rtu in site.system.items():
      #change setpoint according to schedule if required
      if minute in profile.changes:
        zone.setpoints = [x + profile.changes[minute][0] for x in zone.setpoints]
        zone.dr = profile.changes[minute][1]
      #record current temps and hvac states
      rtu.record()
      zone.record()

      #apply cycling strategy
      if zone.dr:
        regular_setpoint_control(zone,rtu)
      else:
        strategy(zone,rtu)

      #change temps
      prevTemp = zone.temp
      zone.temp = prevTemp + \
        zone.walls[0]*(zone.walls[1]-prevTemp) + \
        zone.driftrate_factor*(site.weather.temp - prevTemp) + \
        rtu.heatrate[rtu.heater] + \
        rtu.coolrate[rtu.compressor]

      zone.wallChange(prevTemp)

  if graphs:
    for zone,rtu in site.system.items():
      data = {"Minute": [x for x in range(profile.length)],
              "Zone Temp": zone.history,
              "Wall Temp": zone.wallhistory,
              "Compressor Stage": rtu.history["compressor"],
              "Heater Stage": rtu.history["heater"]}
      df = pd.DataFrame(data)
      fig1 = px.line(df, x=df['Minute'], y=df.columns[1:3])
      fig1.show()
      fig2 = px.bar(df, x=df['Minute'], y=df.columns[3:5])
      fig2.show()

  if energy:
    for zone,rtu in site.system.items():
      return (rtu.energyUsage(), rtu.DRenergy())





In [10]:
def DRcomparison(site, profile, strategy):
  defaultTotal, defaultDR = simulation(site, default)
  strategyTotal, strategyDR = simulation(site, profile, strategy, graphs=True)

  print("Total Energy Usage: " + str(strategyTotal) + "kW   Diff from default: " + str(strategyTotal - defaultTotal) + "kW")
  print("DR energy: " + str(strategyDR) + "kW             Diff from default: " + str(strategyDR - defaultDR) + "kW")


In [12]:
s = site("SITE_NAME")
z = zone("ZONE_NAME")
h = hvac("RTU 1")
s.add_rtu(h,z)
w = weather()
DRcomparison(s,downupdown, regular_setpoint_control)

Total Energy Usage: 175kW   Diff from default: 47kW
DR energy: 35kW             Diff from default: -25kW
